# Set up Azure SQL Database for catching the bus application

This is a SQL Notebook, which allows you to separate text and code blocks and save code results. Azure Data Studio supports several languages, referred to as kernels, including SQL, PowerShell, Python, and more.

In this activity, you'll learn how to import data into Azure SQL Database and create tables to store the route data, geofence data, and real-time bus information.

## Connect to `bus-db`

At the top of the window, select **Select Connection** \> **Change Connection** next to "Attach to".

Under _Recent Connections_ select your `bus-db` connection.

You should now see it listed next to _Attach to_.

## Part 1: Import the bus route data from Azure Blob Storage

The first step in configuring the database for the scenario is to import a CSV file that contains route information data. The following script will walk you through that process. Full documentation on "Accessing data in a CSV file referencing an Azure blob storage location" here: [https://docs.microsoft.com/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage](https://docs.microsoft.com/sql/relational-databases/import-export/examples-of-bulk-access-to-data-in-azure-blob-storage).

You need to first create a table and schema for data to be loaded into.

In [3]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[Routes](
	[Id] [int] NOT NULL,
	[AgencyId] [varchar](100) NULL,
	[ShortName] [varchar](100) NULL,
	[Description] [varchar](1000) NULL,
	[Type] [int] NULL
) ON [PRIMARY]
GO
ALTER TABLE [dbo].[Routes] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.317

The next step is to create a master key.

In [4]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'mySuperStr0n9P@assw0rd!'

Commands completed successfully.

Total execution time: 00:00:00.078

A master key is required to create a `DATABASE SCOPED CREDENTIAL` value because Blob storage is not configured to allow public (anonymous) access. The credential refers to the Blob storage account, and the data portion specifies the container for the store return data.

We use a shared access signature as the identity that Azure SQL knows how to interpret. The secret is the SAS token that you can generate from the Blob storage account. In this example, the SAS token for a storage account that you don't have access to is provided so you can access only the store return data.

In [5]:
CREATE DATABASE SCOPED CREDENTIAL AzureBlobCredentials
WITH IDENTITY = 'SHARED ACCESS SIGNATURE',
SECRET = 'sp=r&st=2021-03-12T00:47:24Z&se=2025-03-11T07:47:24Z&spr=https&sv=2020-02-10&sr=c&sig=BmuxFevKhWgbvo%2Bj8TlLYObjbB7gbvWzQaAgvGcg50c%3D' -- Omit any leading question mark


Commands completed successfully.

Total execution time: 00:00:00.075

Next, create an external data source to the container.

In [6]:
CREATE EXTERNAL DATA SOURCE RouteData
WITH (
	TYPE = blob_storage,
	LOCATION = 'https://azuresqlworkshopsa.blob.core.windows.net/bus',
	CREDENTIAL = AzureBlobCredentials
)

Commands completed successfully.

Total execution time: 00:00:00.074

Now you are ready to bring in the data.

In [7]:
DELETE FROM dbo.[Routes];
INSERT INTO dbo.[Routes]
	([Id], [AgencyId], [ShortName], [Description], [Type])
SELECT 
	[Id], [AgencyId], [ShortName], [Description], [Type]
FROM 
openrowset
	( 
		bulk 'routes.txt', 
		data_source = 'RouteData', 
		formatfile = 'routes.fmt', 
		formatfile_data_source = 'RouteData', 
		firstrow=2,
		format='csv'
	) t;

(0 rows affected)

(132 rows affected)

Total execution time: 00:00:00.239

Finally, let's look at what's been inserted relative to the route we'll be tracking.

In [8]:
SELECT * FROM dbo.[Routes] WHERE [Description] LIKE '%Education Hill%'

(1 row affected)

Total execution time: 00:00:00.083

Id,AgencyId,ShortName,Description,Type
100113,1,221,Education Hill - Crossroads - Eastgate,3


## Part 2: Create necessary tables

### Select a route to monitor

Now that you've added the route information, you can select the route to be a "Monitored Route". This will come in handy if you later choose to monitor multiple routes. For now, you will just add the "Education Hill - Crossroads - Eastgate" route.

In [9]:
-- Create MonitoredRoutes table
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[MonitoredRoutes](
	[RouteId] [int] NOT NULL
) ON [PRIMARY]
GO
ALTER TABLE [dbo].[MonitoredRoutes] ADD PRIMARY KEY CLUSTERED 
(
	[RouteId] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO
ALTER TABLE [dbo].[MonitoredRoutes]  WITH CHECK ADD  CONSTRAINT [FK__MonitoredRoutes__Router] FOREIGN KEY([RouteId])
REFERENCES [dbo].[Routes] ([Id])
GO
ALTER TABLE [dbo].[MonitoredRoutes] CHECK CONSTRAINT [FK__MonitoredRoutes__Router]
GO
-- Monitor the "Education Hill - Crossroads - Eastgate" route
INSERT INTO dbo.[MonitoredRoutes] (RouteId) VALUES (100113);

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.511

### Select a GeoFence to monitor

In addition to monitoring specific bus routes, you will want to monitor certain GeoFences so you can ultimately get notified when your bus enters or exits where you are (i.e. the GeoFence). For now, you will add a small GeoFence that represents the area near the "Crossroads" bus stop.

In [10]:
-- Create GeoFences table
CREATE SEQUENCE [dbo].[global]
    AS INT
    START WITH 1
    INCREMENT BY 1
GO
SELECT  NEXT VALUE FOR [dbo].[global]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[GeoFences](
	[Id] [int] NOT NULL,
	[Name] [nvarchar](100) NOT NULL,
	[GeoFence] [geography] NOT NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
ALTER TABLE [dbo].[GeoFences] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO
ALTER TABLE [dbo].[GeoFences] ADD  DEFAULT (NEXT VALUE FOR [dbo].[global]) FOR [Id]
GO
SET ARITHABORT ON
SET CONCAT_NULL_YIELDS_NULL ON
SET QUOTED_IDENTIFIER ON
SET ANSI_NULLS ON
SET ANSI_PADDING ON
SET ANSI_WARNINGS ON
SET NUMERIC_ROUNDABORT OFF
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[GeoFences]
(
	[GeoFence]
)USING  GEOGRAPHY_AUTO_GRID 
WITH (
CELLS_PER_OBJECT = 12, STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF) ON [PRIMARY]
GO
-- Create a GeoFence
INSERT INTO dbo.[GeoFences] 
	([Name], [GeoFence]) 
VALUES
	('Crossroads', 0xE6100000010407000000B4A78EA822CF4740E8D7539530895EC03837D51CEACE4740E80BFBE630895EC0ECD7DF53EACE4740E81B2C50F0885EC020389F0D03CF4740E99BD2A1F0885EC00CB8BEB203CF4740E9DB04FC23895EC068C132B920CF4740E9DB04FC23895EC0B4A78EA822CF4740E8D7539530895EC001000000020000000001000000FFFFFFFF0000000003);
GO


Commands completed successfully.

(1 row affected)

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:01.013

(No column name)
1


### Create table to track activity in the GeoFence

Next, create a system-versioned table to keep track of what activity is currently happening within the GeoFence. This means tracking buses entering, exiting, and staying within a given GeoFence. Another table within that table will serve as a histroical log for all activity.

In [11]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[GeoFencesActive](
	[Id] [int] IDENTITY(1,1) NOT NULL PRIMARY KEY CLUSTERED,
	[VehicleId] [int] NOT NULL,
	[DirectionId] [int] NOT NULL,
	[GeoFenceId] [int] NOT NULL,
	[SysStartTime] [datetime2](7) GENERATED ALWAYS AS ROW START NOT NULL,
	[SysEndTime] [datetime2](7) GENERATED ALWAYS AS ROW END NOT NULL,
	PERIOD FOR SYSTEM_TIME ([SysStartTime], [SysEndTime])
) ON [PRIMARY]
WITH
(
SYSTEM_VERSIONING = ON ( HISTORY_TABLE = [dbo].[GeoFencesActiveHistory] )
)
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.232

### Create a table to store real-time bus data

You'll need one last table to store the real-time bus data as it comes in.

In [12]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [dbo].[BusData](
	[Id] [int] IDENTITY(1,1) NOT NULL,
	[DirectionId] [int] NOT NULL,
	[RouteId] [int] NOT NULL,
	[VehicleId] [int] NOT NULL,
	[Location] [geography] NOT NULL,
	[TimestampUTC] [datetime2](7) NOT NULL,
	[ReceivedAtUTC] [datetime2](7) NOT NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]
GO
ALTER TABLE [dbo].[BusData] ADD PRIMARY KEY CLUSTERED 
(
	[Id] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO
CREATE NONCLUSTERED INDEX [ix1] ON [dbo].[BusData]
(
	[ReceivedAtUTC] DESC
)WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
GO
ALTER TABLE [dbo].[BusData] ADD  DEFAULT (sysutcdatetime()) FOR [ReceivedAtUTC]
GO
SET ARITHABORT ON
SET CONCAT_NULL_YIELDS_NULL ON
SET QUOTED_IDENTIFIER ON
SET ANSI_NULLS ON
SET ANSI_PADDING ON
SET ANSI_WARNINGS ON
SET NUMERIC_ROUNDABORT OFF
GO
CREATE SPATIAL INDEX [ixsp] ON [dbo].[BusData]
(
	[Location]
)USING  GEOGRAPHY_AUTO_GRID 
WITH (
CELLS_PER_OBJECT = 12, STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF) ON [PRIMARY]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.591

Confirm you've created the tables with the following.

In [14]:
EXEC sp_tables
    @table_owner = 'dbo';

(6 rows affected)

Total execution time: 00:00:00.087

TABLE_QUALIFIER,TABLE_OWNER,TABLE_NAME,TABLE_TYPE,REMARKS
bus-db,dbo,BusData,TABLE,NULL
bus-db,dbo,GeoFences,TABLE,NULL
bus-db,dbo,GeoFencesActive,TABLE,NULL
bus-db,dbo,GeoFencesActiveHistory,TABLE,NULL
bus-db,dbo,MonitoredRoutes,TABLE,NULL
bus-db,dbo,Routes,TABLE,NULL


## Part 3: Create Stored Procedures to get and add data

A stored procedure is a way to group SQL statements and execute them on the database with one command. For the catching the bus scenario, three stored procedures will be required:

1. **web.AddBusData**: this stored procedure takes in JSON data containing new bus route, vehicle, direction, time, and location information and adds it to the _busData_ table. If a bus enters/exits a GeoFence, it will also log this information in the _GeoFencesActive_ table.

In [15]:
CREATE SCHEMA [web]
    AUTHORIZATION [dbo];
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
/*
	Add received Bus geolocation data and check if buses are
	inside any defined GeoFence. JSON must be like:

	{
		"DirectionId": 1,
		"RouteId": 100001,
		"VehicleId": 2,
		"Position": {
			"Latitude": 47.61705102765316,
			"Longitude": -122.14291865504012 
		},
		"TimestampUTC": "20201031"
	}
}
*/
CREATE   procedure [web].[AddBusData]
@payload nvarchar(max) 
as
begin	
	set nocount on
	set xact_abort on
	set tran isolation level serializable
	begin tran

	if (isjson(@payload) != 1) begin;
		throw 50000, 'Payload is not a valid JSON document', 16;
	end;

	declare @ids as table (id int);

	-- insert bus data
	insert into dbo.[BusData] 
		([DirectionId], [RouteId], [VehicleId], [Location], [TimestampUTC])
	output
		[Inserted].Id into @ids
	select
		[DirectionId], 
		[RouteId], 
		[VehicleId], 
		geography::Point([Latitude], [Longitude], 4326) as [Location], 
		[TimestampUTC]
	from
		openjson(@payload) with (
			[DirectionId] int,
			[RouteId] int,
			[VehicleId] int,
			[Latitude] decimal(10,6) '$.Position.Latitude',
			[Longitude] decimal(10,6) '$.Position.Longitude',
			[TimestampUTC] datetime2(7)
		)
		
	-- Get details of inserted data
	select * into #t from dbo.[BusData] where id  in (select i.id from @ids i);

	-- Find geofences in which the vehicle is in
	select 
		t.[Id] as BusDataId,
		t.[VehicleId],
		t.[DirectionId],
		t.[TimestampUTC],
		t.[RouteId],		
		g.Id as GeoFenceId
	into
		#g
	from 
		dbo.GeoFences g 
	right join
		#t t on g.GeoFence.STContains(t.[Location]) = 1;

	-- Calculate status
	select
		c.BusDataId,
		coalesce(a.[GeoFenceId], c.[GeoFenceId]) as GeoFenceId,
		coalesce(a.[DirectionId], c.[DirectionId]) as DirectionId,
		coalesce(a.[VehicleId], c.[VehicleId]) as VehicleId,
		c.[RouteId],
		c.[TimestampUTC],
		case 
			when a.GeoFenceId is null and c.GeoFenceId is not null then 'Enter'
			when a.GeoFenceId is not null and c.GeoFenceId is null then 'Exit'		
		end as [Status]
	into
		#s 
	from
		#g c
	full outer join
		dbo.GeoFencesActive a on c.DirectionId = a.DirectionId and c.VehicleId = a.VehicleId;
	
	-- Delete exited geofences
	delete 
		a
	from
		dbo.GeoFencesActive a
	inner join
		#s s on a.VehicleId = s.VehicleId and s.DirectionId = a.DirectionId and s.[Status] = 'Exit';

	-- Insert entered geofences
	insert into dbo.GeoFencesActive 
		([GeoFenceId], [DirectionId], [VehicleId])
	select
		[GeoFenceId], [DirectionId], [VehicleId]
	from
		#s s
	where 
		s.[Status] = 'Enter';

	-- Insert Log
	insert into dbo.GeoFenceLog 
		(GeoFenceId, BusDataId, [RouteId], [VehicleId], [TimestampUTC], [Status])
	select
		GeoFenceId, BusDataId, [RouteId], [VehicleId], [TimestampUTC], isnull([Status], 'In')
	from
		#s s
	where
		s.[GeoFenceId] is not null
	and
		s.[BusDataId] is not null

	-- Return Entered or Exited geofences
	select
	((
		select
			s.[BusDataId],  
			s.[VehicleId],
			s.[DirectionId],  
			s.[RouteId], 
			r.[ShortName] as RouteName,
			s.[GeoFenceId], 
			gf.[Name] as GeoFence,
			s.[Status] as GeoFenceStatus,
			s.[TimestampUTC]
		from
			#s s
		inner join
			dbo.[GeoFences] gf on s.[GeoFenceId] = gf.[Id]
		inner join
			dbo.[Routes] r on s.[RouteId] = r.[Id]
		where
			s.[Status] is not null and s.[GeoFenceId] is not null
		for 
			json path
	)) as ActivatedGeoFences
	commit
end
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.303

2. **web.GetMonitoredRoutes**: this stored procedure returns the route IDs for the bus routes that are being monitored.

In [16]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

/*
	Return the Routes (and thus the buses) to monitor
*/
CREATE   procedure [web].[GetMonitoredRoutes]
as
begin
	select 
	((	
		select RouteId from dbo.[MonitoredRoutes] for json auto
	)) as MonitoredRoutes
end
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.222

3. **web.GetMonitoredBusData**: this stored procedure will return bus information for the 50 most-recent buses within 5 kilometers of the monitored GeoFence(s).

In [17]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

/*
	Return last geospatial data for bus closest to the GeoFence
*/
CREATE procedure [web].[GetMonitoredBusData]
@routeId int,
@geofenceId int
as
begin
	with cte as
	(
		-- Get the latest location of all the buses in the given route
		select top (1) with ties 
			*  
		from 
			dbo.[BusData] 
		where
			[RouteId] = @routeId
		order by 
			[ReceivedAtUTC] desc
	),
	cte2 as
	(
		-- Get the closest to the GeoFence
		select top (1)
			c.[VehicleId],
			gf.[GeoFence],
			c.[Location].STDistance(gf.[GeoFence]) as d
		from
			[cte] c
		cross join
			dbo.[GeoFences] gf
		where
			gf.[Id] = @geofenceId
		order by
			d 
	), cte3 as
	(
	-- Take the last 50 points 
	select top (50)
		[bd].[VehicleId],
		[bd].[DirectionId],
		[bd].[Location] as l,
		[bd].[Location].STDistance([GeoFence]) as d
	from
		dbo.[BusData] bd
	inner join
		cte2 on [cte2].[VehicleId] = [bd].[VehicleId]
	order by 
		id desc
	)
	-- Return only the points that are withing 5 Km
	select 
	((
		select
			geography::UnionAggregate(l).ToString() as [busData],
			(select [GeoFence].ToString() from dbo.[GeoFences] where Id = @geofenceId) as [geoFence]
		from
			cte3
		where
			d < 5000
		for json auto, include_null_values, without_array_wrapper
	)) as locationData
end
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.228

Confirm you've created the stored procedures with the following.

In [21]:
EXEC sp_stored_procedures
    @sp_owner = 'web';

(3 rows affected)

Total execution time: 00:00:00.080

PROCEDURE_QUALIFIER,PROCEDURE_OWNER,PROCEDURE_NAME,NUM_INPUT_PARAMS,NUM_OUTPUT_PARAMS,NUM_RESULT_SETS,REMARKS,PROCEDURE_TYPE
bus-db,web,AddBusData;1,-1,-1,-1,NULL,2
bus-db,web,GetMonitoredBusData;1,-1,-1,-1,NULL,2
bus-db,web,GetMonitoredRoutes;1,-1,-1,-1,NULL,2
